# <img src="../assets/logo_infnetv1.png" alt="Infnet logo" height="45"/> Projeto de Disciplina de Redes Neurais com TensorFlow
<img src="https://img.shields.io/badge/python-v._3.11.5-blue?style=flat-square&logo=python&logoColor=white" alt="python_logo" height="20"/>
<img src="https://img.shields.io/badge/jupyter-v._5.7.2-blue?style=flat-square&logo=jupyter&logoColor=white" alt="jupyter_logo" height="20"/>
<img src="https://img.shields.io/badge/anaconda-v._23.7.4-blue?style=flat-square&logo=anaconda&logoColor=white" alt="anaconda_logo" height="20"/>

#### Grupo: 

- Mateus Teixeira Ramos da Silva <a href="https://github.com/GitMateusTeixeira/03-ml-modeling"><img src="https://img.shields.io/badge/Github-151b23?style=flat-square&logo=github" alt="github_logo" height="20"/> </a>
- aaaa
- aaaa
- aaaa
- aaaa

### Sobre o projeto:

---

Se trata de um modelo de aprendizagem suprvisionado de classificação binária envolvendo dados relativos ao Câncer de Mama. Os dados foram extraídos do site do <a href="https://www.kaggle.com/datasets/wasiqaliyasir/breast-cancer-dataset">`Kaggle`</a>.

#### Convenções de reprodutibilidade:

- Todas as bibliotecas se encontram no arquivo `📄requirements.txt`
- Para mais informções, consulte nosso `README`

### Índice

---

- <a href='#parte-01-importar-os-pacotes'>Parte 01. Importar os pacotes</a>

- <a href='#parte-02-baixar-e-ler-os-dados'>Parte 02. Baixar e ler os dados</a>

- <a href='#parte-03-análise-exploratória'>Parte 03. Análise exploratória</a>

- <a href='#parte-04-tratamento-dos-dados-exclusão-da-coluna-nula'>Parte 04. Tratamento dos dados</a>

- <a href='#parte-05-verificando-a-correlação-entre-as-colunas'>Parte 05. Verificando a correlação entre as colunas</a>

- <a href='#parte-06-separar-as-features-utilizadas'>Parte 06. Separar as features</a>

- <a href='#parte-07-normalização-com-o-standard-scaller'>Parte 07. Normalização com o Standard Scaller</a>

- <a href='#parte-08-modelos-baseline'>Parte 08. Modelos baseline</a>

- - <a href='#81-criando-um-dumb-model-baseline-prevendo-tudo-como-a-classe-majoritária-para-validações-futuras'>8.1. Dumb model</a>

- - <a href='#82-criando-um-baseline-de-keras-padrão-sem-nenhum-finetuning'>8.2. Baseline com Keras padrão</a>

- <a href='#parte-09-grid-search-com-validação-cruzada'>Parte 09. Grid search com validação cruzada</a>

- <a href='#parte-10-comparação-dos-modelos'>Parte 10. Comparação dos modelos</a>

- <a href='#parte-11-streamlit'>Parte 11. Streamlit</a>

## Parte 01. Importar os pacotes

---

In [ ]:
import kaggle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import tensorflow as tf
import warnings

from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, \
                            confusion_matrix, roc_curve, roc_auc_score, auc, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from scikeras.wrappers import KerasClassifier
from tqdm.auto import tqdm


SEED = 42

pd.set_option("display.max_columns", 100)

# Silenciar os avisos do código
warnings.filterwarnings('ignore')

## Parte 02. Baixar e ler os dados

---

<a href='#índice'>Voltar ao início</a>

O arquivo será baixado diretamente do repositório do Kaggle. Caso a estrutura de pastas não exista, o algoritmo irá construí-la.

In [ ]:
# Inicializar a API do Kaggle
api = KaggleApi()
api.authenticate()

# Criar o diretório se não existir
data_path = '../data/01-raw'
os.makedirs(data_path, exist_ok=True)

# Baixar os dados do Kaggle
api.dataset_download_files('wasiqaliyasir/breast-cancer-dataset', path=data_path, unzip=True)

In [ ]:
path_raw = '../data/01-raw'
file_raw = 'Breast_cancer_dataset.csv'

pathfile_raw = os.path.join(path_raw, file_raw)

In [ ]:
df = pd.read_csv(pathfile_raw)

## Parte 03. Análise exploratória

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
print(f"O dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas.")

### 3.1. Visão geral do dataset

In [ ]:
# Detalhando a base de dados com moda
def check(df):
    l = []
    colunas = df.columns
    
    for col in colunas:
        dtypes = df[col].dtypes
        nunique = df[col].nunique()
        sum_null = df[col].isnull().sum()

        # Calcula a moda e a frequência da moda
        moda = df[col].mode().iloc[0] if not df[col].mode().empty else "Não se aplica"
        moda_freq = df[col].value_counts().iloc[0] if not df[col].value_counts().empty else "Não se aplica"

        if np.issubdtype(dtypes, np.number):
            status = df.describe(include='all').T
            media = status.loc[col, 'mean']
            std = status.loc[col, 'std']
            min_val = status.loc[col, 'min']
            quar1 = status.loc[col, '25%']
            mediana = df[col].median()
            quar3 = status.loc[col, '75%']
            max_val = status.loc[col, 'max']
                    
        else:
            status = "Não se aplica"
            media = "Não se aplica"
            std = "Não se aplica"
            min_val = "Não se aplica"
            quar1 = "Não se aplica"
            mediana = "Não se aplica"
            quar3 = "Não se aplica"
            max_val = "Não se aplica"

        l.append([col, dtypes, nunique, sum_null, media, std, min_val, quar1, mediana, quar3, max_val, moda, moda_freq])
    
    # Criação do DataFrame com as novas colunas
    df_check = pd.DataFrame(l)
    df_check.columns = ['coluna', 'tipo', 'únicos', 'null_soma', 'media', 'desvio', 
                        'minimo', '25%', 'mediana', '75%', 'maximo', 'moda', 'frequência_moda']
    
    return df_check 

In [ ]:
# Análise geral dos dados
check(df)

In [ ]:
# Análise dos tipos das colunas
df.info()

É possível verificar que o dataset original possui 33 colunas, das quais uma ('Unamed: 32') possui apenas dados nulos.

Além disso, com execeção da coluna 'diagnosis' (que é uma coluna categórica), as demais colunas são colunas numéricas, com informações sobre cada paciente.

### 3.2. Análise da distribuição dos dados

### 3.3. Pairplot - Análise de Relacionamentos entre Variáveis

O **Pairplot** (ou scatterplot matrix) é uma ferramenta fundamental na análise exploratória de dados (EDA). 
Este gráfico mostra, para cada par de variáveis numéricas, como elas se relacionam, além da distribuição individual 
em forma de histograma/densidade na diagonal.

**Para que serve:**

- **Exploração de dados (EDA)**: Antes de treinar uma rede neural (ou qualquer modelo de machine learning), 
  é essencial entender como os dados estão distribuídos, se existem correlações fortes entre variáveis e 
  se há separabilidade entre classes.

- **Redução de dimensionalidade / seleção de features**: O pairplot ajuda a identificar variáveis altamente 
  correlacionadas (como radius_mean, perimeter_mean e area_mean), que podem ser redundantes. Isso é útil 
  porque redes neurais podem sofrer com dados redundantes ou multicolinearidade.

- **Identificação de padrões de separação**: Você pode ver se as classes se separam visualmente em 
  determinados pares de features. Isso dá uma intuição de quais variáveis carregam mais poder discriminativo.

**Quando usar em relação a uma rede neural:**

- **Antes do treinamento**: Para inspecionar os dados, escolher features relevantes e entender possíveis 
  ajustes de pré-processamento (normalização, remoção de redundâncias).

- **Não durante o treinamento**: O pairplot é puramente exploratório; não entra como input em uma rede neural. 
  A rede usará os valores numéricos das features (normalizados ou padronizados), não o gráfico em si.

**Fluxo típico:**

1. EDA com pairplot → 2. Pré-processamento (scaling, seleção de features, balanceamento de classes) → 
3. Treino da rede neural (ou outro modelo).

Este gráfico é como o 'raio-X' inicial dos dados, antes de colocar a rede para aprender.

In [ ]:
# Primeiro, vamos criar uma coluna numérica para o diagnóstico
df_pairplot = df.copy()

# Codificar a variável target
label_encoder = LabelEncoder()
df_pairplot['diagnosis_encoded'] = label_encoder.fit_transform(df_pairplot['diagnosis'])

# Selecionar algumas variáveis principais para o pairplot (para não sobrecarregar o gráfico)
features_for_pairplot = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'diagnosis_encoded']

# Criar o subset dos dados
df_subset = df_pairplot[features_for_pairplot]

print(f"Variáveis selecionadas para o Pairplot: {features_for_pairplot[:-1]}")
print(f"Classes: 0 = Benigno ({sum(df_pairplot['diagnosis_encoded'] == 0)} casos), 1 = Maligno ({sum(df_pairplot['diagnosis_encoded'] == 1)} casos)")

In [ ]:
# Configurar o estilo do seaborn
plt.style.use('default')
sns.set_palette("husl")

# Criar o pairplot
fig = plt.figure(figsize=(15, 12))

# Pairplot com separação por classe
pairplot = sns.pairplot(
    df_subset, 
    hue='diagnosis_encoded',
    diag_kind='hist',
    plot_kws={'alpha': 0.6, 's': 30},
    diag_kws={'alpha': 0.7}
)

# Personalizar o gráfico
pairplot.fig.suptitle('Pairplot - Análise de Relacionamentos entre Variáveis do Câncer de Mama', 
                      fontsize=16, y=1.02)

# Ajustar as legendas
handles = pairplot._legend_data.values()
labels = ['Benigno (0)', 'Maligno (1)']
pairplot.fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.98, 0.98))

# Remover a legenda original
pairplot._legend.remove()

plt.tight_layout()
plt.show()

**Análise do Pairplot:**

1. **Correlações Fortes**: Observamos correlações muito fortes entre `radius_mean`, `perimeter_mean` e `area_mean`, 
   o que é esperado geometricamente (raio, perímetro e área estão matematicamente relacionados).

2. **Separabilidade das Classes**: 
   - Tumores malignos (classe 1) tendem a ter valores maiores de raio, perímetro e área
   - Existe uma boa separação visual entre as classes em várias combinações de variáveis
   - A textura também mostra alguma capacidade discriminativa

3. **Distribuições**: 
   - As distribuições na diagonal mostram que algumas variáveis podem se beneficiar de normalização
   - Há evidência de que as classes têm distribuições diferentes para a maioria das variáveis

4. **Implicações para a Rede Neural**:
   - A forte correlação entre algumas variáveis sugere que podemos considerar redução de dimensionalidade
   - A boa separabilidade visual indica que uma rede neural deve conseguir aprender padrões discriminativos
   - A necessidade de normalização é evidente devido às diferentes escalas das variáveis

É possível verificar que o dataset original possui 33 colunas, das quais uma ('Unamed: 32') possui apenas dados nulos.

Além disso, com execeção da coluna 'diagnosis' (que é uma coluna categórica), as demais colunas são colunas numéricas, com informações sobre cada paciente.

### 3.2. Análise da distribuição dos dados

In [ ]:
# Análise da distribuição da variável target
print(f"Distribuição da variável target:")
print(df['diagnosis'].value_counts())
print(f"\nPercentual da distribuição da variável target:")
print(df['diagnosis'].value_counts(normalize=True) * 100)

In [ ]:
# Visualização da distribuição da variável target
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='diagnosis', palette='viridis')
plt.title('Distribuição da Variável Target (Diagnosis)')
plt.xlabel('Diagnóstico')
plt.ylabel('Frequência')
plt.show()

## Parte 04. Tratamento dos dados (exclusão da coluna nula)

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
# Removendo a coluna com valores nulos
df = df.drop('Unnamed: 32', axis=1)
print(f"Novo shape do dataset: {df.shape}")

## Parte 05. Verificando a correlação entre as colunas

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
# Codificando a variável target para análise de correlação
le = LabelEncoder()
df['diagnosis_encoded'] = le.fit_transform(df['diagnosis'])
print(f"Mapeamento: {dict(zip(le.classes_, le.transform(le.classes_)))}")

In [ ]:
# Matriz de correlação
plt.figure(figsize=(20, 16))
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de Correlação das Variáveis Numéricas')
plt.tight_layout()
plt.show()

## Parte 06. Separar as features utilizadas

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
# Separando features e target
X = df.drop(['id', 'diagnosis', 'diagnosis_encoded'], axis=1)
y = df['diagnosis_encoded']

print(f"Shape das features (X): {X.shape}")
print(f"Shape do target (y): {y.shape}")

## Parte 07. Normalização com o Standard Scaler

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)

print(f"Shape X_train: {X_train.shape}")
print(f"Shape X_test: {X_test.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape y_test: {y_test.shape}")

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Média antes da normalização: {X_train.mean().mean():.4f}")
print(f"Desvio padrão antes da normalização: {X_train.std().mean():.4f}")
print(f"Média após a normalização: {X_train_scaled.mean():.4f}")
print(f"Desvio padrão após a normalização: {X_train_scaled.std():.4f}")

## Parte 08. Modelos baseline

---

<a href='#índice'>Voltar ao início</a>

### 8.1. Criando um dumb model (baseline) prevendo tudo como a classe majoritária para validações futuras

In [ ]:
# Dumb model - sempre prediz a classe majoritária
from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier(strategy='most_frequent', random_state=SEED)
dummy_model.fit(X_train_scaled, y_train)

# Predições
y_pred_dummy = dummy_model.predict(X_test_scaled)

# Métricas
print("=== DUMB MODEL (Baseline) ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dummy):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_dummy):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_dummy):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_dummy):.4f}")

### 8.2. Criando um baseline de Keras padrão (sem nenhum fine-tuning)

In [ ]:
# Modelo baseline com Keras
def create_baseline_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Criando e treinando o modelo
baseline_model = create_baseline_model()

# Treinamento
history_baseline = baseline_model.fit(X_train_scaled, y_train,
                                     epochs=100,
                                     batch_size=32,
                                     validation_split=0.2,
                                     verbose=0)

# Predições
y_pred_baseline_prob = baseline_model.predict(X_test_scaled)
y_pred_baseline = (y_pred_baseline_prob > 0.5).astype(int).flatten()

# Métricas
print("=== KERAS BASELINE MODEL ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_baseline):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_baseline):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_baseline):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_baseline):.4f}")

## Parte 09. Grid search com validação cruzada

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
# Função para criar modelo para grid search
def create_model(neurons=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dropout(dropout_rate),
        Dense(neurons//2, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
# Grid search com validação cruzada
keras_classifier = KerasClassifier(model=create_model, epochs=50, batch_size=32, verbose=0)

# Parâmetros para grid search
param_grid = {
    'model__neurons': [32, 64, 128],
    'model__learning_rate': [0.001, 0.01],
    'model__dropout_rate': [0.1, 0.2, 0.3]
}

# Grid search
grid_search = GridSearchCV(estimator=keras_classifier,
                          param_grid=param_grid,
                          cv=3,
                          scoring='f1',
                          n_jobs=-1,
                          verbose=1)

# Executar grid search
print("Executando Grid Search...")
grid_result = grid_search.fit(X_train_scaled, y_train)

# Melhores parâmetros
print(f"Melhores parâmetros: {grid_result.best_params_}")
print(f"Melhor score: {grid_result.best_score_:.4f}")

In [ ]:
# Avaliação do melhor modelo
best_model = grid_result.best_estimator_

# Predições
y_pred_best_prob = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_best = best_model.predict(X_test_scaled)

# Métricas
print("=== MELHOR MODELO (Grid Search) ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_best):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_best):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_best_prob):.4f}")

## Parte 10. Comparação dos modelos

---

<a href='#índice'>Voltar ao início</a>

In [ ]:
# Comparação dos modelos
models_comparison = pd.DataFrame({
    'Modelo': ['Dumb Classifier', 'Keras Baseline', 'Melhor Modelo (Grid Search)'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_dummy),
        accuracy_score(y_test, y_pred_baseline),
        accuracy_score(y_test, y_pred_best)
    ],
    'Precision': [
        precision_score(y_test, y_pred_dummy),
        precision_score(y_test, y_pred_baseline),
        precision_score(y_test, y_pred_best)
    ],
    'Recall': [
        recall_score(y_test, y_pred_dummy),
        recall_score(y_test, y_pred_baseline),
        recall_score(y_test, y_pred_best)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_dummy),
        f1_score(y_test, y_pred_baseline),
        f1_score(y_test, y_pred_best)
    ]
})

print("=== COMPARAÇÃO DOS MODELOS ===")
print(models_comparison.round(4))

In [ ]:
# Matrizes de confusão
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

models_preds = [y_pred_dummy, y_pred_baseline, y_pred_best]
model_names = ['Dumb Classifier', 'Keras Baseline', 'Melhor Modelo']

for i, (pred, name) in enumerate(zip(models_preds, model_names)):
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'Matriz de Confusão - {name}')
    axes[i].set_xlabel('Predito')
    axes[i].set_ylabel('Real')

plt.tight_layout()
plt.show()

In [ ]:
# Curvas ROC
plt.figure(figsize=(10, 8))

# ROC para baseline
fpr_baseline, tpr_baseline, _ = roc_curve(y_test, y_pred_baseline_prob.flatten())
auc_baseline = auc(fpr_baseline, tpr_baseline)

# ROC para melhor modelo
fpr_best, tpr_best, _ = roc_curve(y_test, y_pred_best_prob)
auc_best = auc(fpr_best, tpr_best)

# Plot
plt.plot(fpr_baseline, tpr_baseline, label=f'Keras Baseline (AUC = {auc_baseline:.3f})', linewidth=2)
plt.plot(fpr_best, tpr_best, label=f'Melhor Modelo (AUC = {auc_best:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Linha de Base (AUC = 0.5)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curvas ROC - Comparação dos Modelos')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

## Parte 11. Streamlit

---

<a href='#índice'>Voltar ao início</a>

*Nota: A implementação do Streamlit será feita apenas na pipeline de produção.*